<img src="https://pbs.twimg.com/media/E0m6c9FX0AIX0p1.png" style="height:150px" align="left"/> <br><br>

<br/>
<br/>
<br/>
<br/>
<br/>

### Common data cleaning methods

***

The GEOROC Database (Geochemistry of Rocks of the Oceans and Continents) is a comprehensive collection of published analyses <br/> of igneous and metamorphic rocks and minerals. It contains major and trace element concentrations, radiogenic and nonradiogenic <br/> isotope ratios as well as analytical ages for whole rocks, glasses, minerals and inclusions. Metadata include geospatial and other <br/> sample information, analytical details and references.

The GEOROC Database was established at the Max Planck Institute for Chemistry in Mainz (Germany). In 2021, the database was <br/> moved to Göttingen University, where it continues to be curated as part of the DIGIS project of the Department of Geochemistry and <br/> Isotope Geology at the Geoscience Centre (GZG) and the University and State Library (SUB). Development for GEOROC 2.0 <br/> includes a new data model for greater interoperability, options to contribute data, and improved access to the database.

As part of the DIGIS project, a new API interface has been created for the GEOROC database, allowing easy access to its contents <br/>
with simple programming skills. Users can now query the database and retrieve data using the new API, making it more accessible <br/>
and useful for researchers and other interested parties. This notebook demonstrates the basic capabilities of GEOROC data access <br/> via the new DIGIS API. 

For feedback, questions and further information contact [Digis-Info](mailto:digis-info@uni-goettingen.de) directly.

This notebook demonstrates the basic capabilities of common data cleaning methods in geoscience.

***

April 2023, Dr. Marthe Klöcking, Timm M. Wirtz

<img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" style="height:50px" align="left"/> <br><br>

https://creativecommons.org/licenses/by-nc-sa/4.0/

***

### <a name="table"></a>Table of contents

* [Installing additional software and modules](#install)
        * [Hints](#hints)
    * [Change the Kernel](#Kernel)
* [Making API requests to access the database](#request)
        * [Hint](#hint)
    * [Store your API-Key here](#yourkey)
            * [Hint](#hint2)
    * [API call with messages](#apic)
    * [Multiple filter function](#ffunc)
    * [Extract data from JSON function](#dataf)
    * [Choose your parameters for the filters](#params)
            * [Hint](#hint3)
    * [Extract SamplingFeatureIDs](#esfid)
    * [Extract measurement data](#emdata)
* [Plotting the extracted data on a map](#plot)
* [References](#refs)
* [QR code for accessing this Jupyter notebook](#qrcode)

***

### <a name="install"></a>Installing additional software and modules

##### <a name="hints"></a>Hint:
- For terminal commands in JupyterLab etc always at the beginning with a ! symbol. Otherwise not.

In [ ]:
!pip install modul_x

***

Import modules

In [ ]:
import requests
import json
import io
from contextlib import redirect_stdout
import pandas as pd
from tqdm.auto import tqdm
from ipywidgets import widgets

***

In [ ]:
# Erstellen Sie ein Text-Widget
api_key_widget = widgets.Text(
    placeholder='Geben Sie Ihren API-Schlüssel ein',
    description='API Schlüssel:',
    disabled=False,
    layout=Layout(width="auto"
)

# Anzeige des Widgets
display(api_key_widget);

In [ ]:
# Define API-Key, Base-URL and headers
api_key = api_key_widget.value
base_url = "https://api-test.georoc.eu/api/v1/"
headers = {
    "accept": "application/json",
    "DIGIS-API-ACCESSKEY": api_key
}

In [ ]:
def api_query(endpoint, params=None):
    response = requests.get(f"{base_url}{endpoint}", headers=headers, params=params)
    if response.status_code == 200:
        data = json.loads(response.text)
        print("\n", f"API query successful for endpoint:{endpoint}", "\n")
        return data
    else:
        print(f"API query error for endpoint: {endpoint}, Statuscode:{response.status_code}", "\n")
        print(response.text)
        return None

In [ ]:
def get_filtered_samples(
    limit=None,
    offset=None,
    setting=None,
    location1=None,
    location2=None,
    location3=None,
    latitude=None,
    longitude=None,
    rocktype=None,
    rockclass=None,
    mineral=None,
    material=None,
    inclusiontype=None,
    sampletech=None,
    element=None,
    elementtype=None,
    value=None,
    title=None,
    publicationyear=None,
    doi=None,
    firstname=None,
    lastname=None,
    agemin=None,
    agemax=None,
    geoage=None,
    geoageprefix=None,
    lab=None
):
    endpoint = "queries/samples"

    filters = {
        key: value
        for key, value in locals().items()
        if key not in ["endpoint"] and value is not None
    }

    data = api_query(endpoint, params=filters)
    return data

# Specify the desired keys
keys = ['Item_Name', 'Values', 'Units', 'Item_Group']

In [ ]:
def get_measurement_data(api_key, sampling_feature_id):
    base_url = f"https://api-test.georoc.eu/api/v1/queries/fulldata/{sampling_feature_id}"
    headers = {
        "accept": "application/json",
        "DIGIS-API-ACCESSKEY": api_key
    }

    response = requests.get(base_url, headers=headers)
    if response.status_code == 200:
        data = json.loads(response.text)['Data'][0]

        # Create a dictionary containing all keys and values from data
        df_data = {key: data[key] for key in keys}

        # Convert all values in df_data to lists
        for key, value in df_data.items():
            if not isinstance(value, list):
                df_data[key] = [value]

        # Find the longest list in df_data
        max_len = max([len(value) for value in df_data.values()])

        # Extend all the lists to the length of the longest list
        for key, value in df_data.items():
            if len(value) < max_len:
                df_data[key] = value + [None] * (max_len - len(value))

        # Create the DataFrame
        df = pd.DataFrame(df_data)

        return df

    else:
        print(f"Error fetching data for sample ID {sampling_feature_id}, Status code: {response.status_code}")
        return None

In [ ]:
def check_api_connection():
    endpoint = "ping"
    response = api_query(endpoint)

    if response is not None:
        print("Connection to API server successful!\n")
    else:
        print("Failed to connect to API server!\n")
        exit(1)


# Check the connection to the API server
check_api_connection()

In [ ]:
# Get SamplingFeatureIDs for the given filters
filtered_samples_combined = get_filtered_samples(
    limit="1",  # update the limit to retrieve more than one SamplingFeatureID
    location1="EAST AFRICAN RIFT",
    location2="ETHIOPIAN RIFT",
    setting="RIFT VOLCANICS"
)

print(filtered_samples_combined, "\n")

In [ ]:
# Extract SamplingFeatureIDs
if "Data" in filtered_samples_combined and filtered_samples_combined["Data"]:
    sampling_feature_ids = [sample["SampleID"] for sample in filtered_samples_combined["Data"]]
    print("\n", f"The extracted SampleIDs are:", sampling_feature_ids, "\n")
else:
    print("No data found or unexpected data structure", "\n")

In [ ]:
# Create an empty DataFrame to store all measurement data
measurement_data = pd.DataFrame()

In [ ]:
# Initialize the progress bar
progress_bar = tqdm(
    total=len(sampling_feature_ids),
    desc="Processing",
    unit="sample",
    dynamic_ncols=True,
    leave=True,
    colour="#00507d",
)

# Redirect the standard output to suppress intermediate prints
with io.StringIO() as buf, redirect_stdout(buf):

    # Iterate over the list of SamplingFeatureIDs
    for sampling_feature_id in sampling_feature_ids:
        print(f"Fetching measurement data for SamplingFeatureID: {sampling_feature_id}")

        # Get the measurement data for the current SamplingFeatureID
        df = get_measurement_data(api_key, sampling_feature_id)

        # Check if the dataframe is not empty and not None
        if df is not None and not df.empty:
            # Append the dataframe to the measurement_data DataFrame
            measurement_data = measurement_data.append(df, ignore_index=True)
        elif df is None:
            print(f"Error occurred while fetching data for SamplingFeatureID {sampling_feature_id}")
        else:
            print(f"No measurement data found for SamplingFeatureID {sampling_feature_id}")

        # Update the progress bar for each completed step
        progress_bar.update(1)

# Close the progress bar after the loop
progress_bar.close()

In [ ]:
# Save the combined measurement_data DataFrame
measurement_data.to_csv('measurement_data.csv', index=False)

### <a name="refs"></a>References

[1] GMT 6: Wessel, P., Luis, J. F., Uieda, L., Scharroo, R., Wobbe, F., Smith, W. H. F., & Tian, D. (2019). The Generic Mapping Tools version 6. Geochemistry, Geophysics, Geosystems, 20, 5556–5564. https://doi.org/10.1029/2019GC008515

[2] Uieda, Leonardo, Tian, Dongdong, Leong, Wei Ji, Schlitzer, William, Grund, Michael, Jones, Max, Fröhlich, Yvonne, Toney, Liam, Yao, Jiayuan, Magen, Yohai, Jing-Hui, Tong, Materna, Kathryn, Belem, Andre, Newton, Tyler, Anant, Abhishek, Ziebarth, Malte, Quinn, Jamie, & Wessel, Paul. (2023). PyGMT: A Python interface for the Generic Mapping Tools (v0.9.0). Zenodo. https://doi.org/10.5281/zenodo.7772533

[3]  Thomas A Caswell, Antony Lee, Elliott Sales de Andrade, Michael Droettboom, Tim Hoffmann, Jody Klymak, John Hunter, Eric Firing, David Stansby, Nelle Varoquaux, Jens Hedegaard Nielsen, Benjamin Root, Ryan May, Oscar Gustafsson, Phil Elson, Jouni K. Seppänen, Jae-Joon Lee, Darren Dale, hannah, … Charlie Moad. (2023). matplotlib/matplotlib: REL: v3.7.1 (v3.7.1). Zenodo. https://doi.org/10.5281/zenodo.7697899

[4] Anenburg, M., & Williams, M. J. (2021). Quantifying the Tetrad Effect, Shape Components, and Ce–Eu–Gd Anomalies in Rare Earth Element Patterns. Mathematical Geosciences. doi: https://doi.org/10.1007/s11004-021-09959-5

[5] The pandas development team. (2023). pandas-dev/pandas: Pandas (v2.0.0). Zenodo. https://doi.org/10.5281/zenodo.7794821

[6] Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). https://doi.org/10.1038/s41586-020-2649-2

***

### <a name="qrcode"></a>QR code for accessing this Jupyter notebook on Binder

<img src="https://raw.githubusercontent.com/tmwProjects/Georoc_jupyter/main/BINDER_JUPYTER_QR.png" style="height:610px" align="left"/> <br><br>

##### <a name="hint"></a>Hint:
- In some cases, Binder may not work the first time it is started. Then simply restart the link again.

[Back to Table of contects](#table)

***

### <a name="follow"></a>Follow us

[Visit our Website](https://georoc.mpch-mainz.gwdg.de/georoc/)

![Twitter Follow](https://img.shields.io/twitter/follow/DIGISgeo?style=social)

[Back to Table of contects](#table)

***